In [1]:
import sys
import os
import yaml
import argparse
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(message)s')

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
import torch

from Networks.Embedding.embedding import Embedding

/users/ang.li/anaconda3/envs/exatrkx-cpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:Created a temporary directory at /tmp/tmp0x60j_r3
INFO:Writing /tmp/tmp0x60j_r3/_remote_module_non_scriptable.py
INFO:Loading faiss with AVX512 support.
INFO:Successfully loaded faiss with AVX512 support.


In [2]:
# Load config file
with open('emb_config.yaml') as c:
    cl = yaml.load(c, Loader=yaml.FullLoader)
    config = cl['metric_learning_configs']

In [3]:
config

{'input_dir': 'Datasets',
 'output_dir': 'Datasets/EMB_output',
 'pt_signal_cut': 1.0,
 'pt_background_cut': 1.0,
 'split': [1000, 92, 91],
 'true_edges': 'modulewise_true_edges',
 'input_dim': 2,
 'hidden_dim': 128,
 'n_layers': 3,
 'emb_dim': 4,
 'seed': 1,
 'lr': 0.001,
 'factor': 0.7,
 'patience': 4,
 'margin': 0.1,
 'activation': 'Tanh',
 'weight': 1,
 'r_max': 0.1,
 'r_val': 0.1,
 'r_test': 0.1,
 'k_max': 20,
 'query_points': 'signal',
 'points_per_batch': 100000,
 'max_epochs': 2}

In [4]:
model = Embedding(config)

In [5]:
save_directory = os.path.join("EMB_out")
logger = CSVLogger(save_directory, name='testrun')
trainer = Trainer(
    accelerator='gpu' if torch.cuda.is_available() else None,
    gpus=0,
    max_epochs=config["max_epochs"],
    logger=logger
)

/users/ang.li/anaconda3/envs/exatrkx-cpu/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:474: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
/users/ang.li/anaconda3/envs/exatrkx-cpu/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /users/ang.li/anaconda3/envs/exatrkx-cpu/lib/python3 ...
  rank_zero_warn(
INFO:GPU available: False, used: False
INFO:TPU available: False, using: 0 TPU cores
INFO:IPU available: False, using: 0 IPUs
INFO:HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model)
os.makedirs(save_directory, exist_ok=True)
trainer.save_checkpoint(os.path.join(save_directory, "testrun.ckpt"))

INFO:
  | Name    | Type       | Params
---------------------------------------
0 | network | Sequential | 34.7 K
---------------------------------------
34.7 K    Trainable params
0         Non-trainable params
34.7 K    Total params
0.139     Total estimated model params size (MB)


Epoch 0:  92%|██████████████████████████▌  | 1000/1092 [00:18<00:01, 53.54it/s, loss=0.01, v_num=10]
Validation: 0it [00:00, ?it/s]
Epoch 1:  92%|██████████████████████████▌  | 1000/1092 [00:18<00:01, 53.67it/s, loss=0.01, v_num=10]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|█████████████████████████████| 1092/1092 [00:19<00:00, 54.74it/s, loss=0.01, v_num=10]

INFO:`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|█████████████████████████████| 1092/1092 [00:19<00:00, 54.66it/s, loss=0.01, v_num=10]
